In [1]:
import os
from tqdm import tqdm

import cv2
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from utils.dataloader import DataLoader
from utils.vis import MatplotlibVisualizer
from utils.transforms import HairRemoval, Composer
from descriptors.texture import LBPDescriptor, GLCMDescriptor
from utils.segmentation import ThresholdingSegmentation
from descriptors.bow import MultifeatureBoW
from descriptors.color import ColorDescriptor, ColorLayoutDescriptor, ColorCooccurrenceMatrixDescriptor

In [2]:
## Classes
CLASSES = ['nevus', 'others']

## Work folfer
work_folder = os.getcwd()
data_folder = os.path.join(work_folder, '..', 'Data/Challenge1')

## Visualizer
matplotlib_visualizer = MatplotlibVisualizer()

In [3]:
color_descriptor = ColorDescriptor(bins=(8, 12, 3))
color_cooccurrence_matrix_descriptor = ColorCooccurrenceMatrixDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8)
# glcm_descriptor = GLCMDescriptor(distances=[1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4], levels=8, visualize=False, grid_x=4, grid_y=4)
# lbp_descriptor = LBPDescriptor(radius=3, n_points=16, grid_x=3, grid_y=3, visualize=False)

descriptors = [color_descriptor, color_cooccurrence_matrix_descriptor]
classifier = xgb.XGBClassifier(objective='binary:logistic', n_estimators=50, learning_rate=0.1, n_jobs=-1)

# Initialize the MultifeatureBoW model
multifeature_bow = MultifeatureBoW(vocab_size=100, descriptors=descriptors)

In [ ]:
train_dataloader = DataLoader(data_folder, 'train', 
                        shuffle=True, 
                        ignore_folders=['black_background', '.DS_Store'], 
                        max_samples=None, 
                        balance=False,
                        transforms=None)

# Build the codebook using the training data
multifeature_bow.build_codebook(train_dataloader)


# Train the classifier using BoW histograms
multifeature_bow.fit_classifier(train_dataloader)

In [24]:
test_dataloader = DataLoader(data_folder, 'val', 
                        shuffle=True, 
                        ignore_folders=['black_background', '.DS_Store'], 
                        max_samples=None, 
                        balance=False,
                        transforms=None)

report, confusion_m = multifeature_bow.evaluate(test_dataloader, class_names=CLASSES)
print(report)
print(confusion_m)

Evaluating...
Predicting...


Transforming Images: 100%|██████████| 2635/2635 [01:24<00:00, 31.12it/s]


              precision    recall  f1-score   support

       nevus       0.63      0.56      0.59      1052
      others       0.73      0.78      0.75      1583

    accuracy                           0.69      2635
   macro avg       0.68      0.67      0.67      2635
weighted avg       0.69      0.69      0.69      2635

[[ 584  468]
 [ 347 1236]]
